In [2]:
import psycopg2

import sys
import os
import random
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader


from Dataset.AutoEncoder_Dataset import AutoEncoder_Dataset
from Model.AutoEncoder import AutoEncoder

from Dataset.LSTM_Dataset import LSTM_Dataset
from Model.LSTM import LSTM



SEED = 1234
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
connection_info = "host=localhost dbname=postgres user=postgres password=hd219833 port=5432"
conn = psycopg2.connect(connection_info)
table_1_query = '''
    SELECT * FROM building
    '''
table_2_query = '''
    SELECT * FROM economy
    '''
table_3_query = '''
    SELECT * FROM building_price
    '''
table_1 = pd.read_sql(table_1_query,conn) 
table_2 = pd.read_sql(table_2_query,conn)
table_3 = pd.read_sql(table_3_query,conn) 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [47]:
embedding_lr = 0.01
embedding_batch = 32
embedding_epochs = 50
embedding_dim = 6

lstm_lr = 0.01
lstm_batch = 1
lstm_epochs = 50
lstm_hidden_dim = 128
lstm_output_dim = 1
lstm_window_size = 5

# valid_every = 5
# best_valid_loss = float('inf')

## LSTM

In [5]:
model = torch.load('../데이터/Checkpoint/embedding_batch_32_epochs_50_dim_6.pth')

apartment_complexes_embedding_matrix_with_window_size = []
apartment_complexes_price_with_window_size = []

apartment_complexes_location = table_1['location']
apartment_complexes_name = table_1['name']
for apartment_complex_location, apartment_complex_name in zip(apartment_complexes_location, apartment_complexes_name):
    apartment_complex_values = table_1[(table_1['name'] == apartment_complex_name) * (table_1['location'] == apartment_complex_location)][[cols for cols in table_1.columns if cols not in ['aid','location','name']]].values
    apartment_complex_tensor = torch.FloatTensor(apartment_complex_values).repeat(204, 1)
    economy_values = table_2[['call_rate','m2']].values
    economy_tensor = torch.FloatTensor(economy_values)
    encoder_input_tensor = torch.cat((apartment_complex_tensor, economy_tensor), dim=1)

    apartment_complex_embedding_matrix = np.zeros((encoder_input_tensor.shape[0], embedding_dim))
    with torch.no_grad():
        for i in range(encoder_input_tensor.shape[0]):
            apartment_complex_embedding_vector = model.encoder(encoder_input_tensor[i].unsqueeze(0)).squeeze().numpy()
            apartment_complex_embedding_matrix[i] = apartment_complex_embedding_vector
    apartment_complex_embedding_matrix_tensor = torch.FloatTensor(apartment_complex_embedding_matrix)

    apartment_complex_aid = table_1[(table_1['name'] == apartment_complex_name) * (table_1['location'] == apartment_complex_location)]['aid'].squeeze()
    price_values = pd.DataFrame({'did': range(0, 204)}).merge(table_3[table_3['aid'] == apartment_complex_aid][['did','price']], on='did', how='outer').fillna(0).set_index('did').values
    price_tensor = torch.FloatTensor(price_values)

    for i in range(apartment_complex_embedding_matrix_tensor.shape[0]-lstm_window_size):
        apartment_complexes_embedding_matrix_with_window_size.append(apartment_complex_embedding_matrix_tensor[i:i+lstm_window_size, :])
        apartment_complexes_price_with_window_size.append(price_tensor[i+lstm_window_size, :])

apartment_complexes_embedding_matrix_with_window_size = apartment_complexes_embedding_matrix_with_window_size
apartment_complexes_price_with_window_size = apartment_complexes_price_with_window_size

In [46]:
apartment_complexes_embedding_matrix_with_window_size[0].shape

torch.Size([5, 6])

## Attention

In [50]:
table_1['dong'] = table_1['location'].apply(lambda x: x.split(' ')[2])
dongs = table_1['dong'].unique()
max_dong_apartment_complexes = 38

In [75]:
for dong in dongs:
    dong_apartment_complexes_values = table_1[table_1['dong'] == dong][[cols for cols in table_1.columns if cols not in ['aid','location','name','dong']]].values
    economy_values = table_2[['call_rate','m2']].values
    economy_tensor = torch.FloatTensor(economy_values)

    encoder_input_tensors = torch.zeros(dong_apartment_complexes_values.shape[0],204,12)
    for i, dong_apartment_complex_values in enumerate(dong_apartment_complexes_values):
        dong_apartment_complex_tensor = torch.FloatTensor(dong_apartment_complex_values).repeat(204,1)
        encoder_input_tensor = torch.cat((dong_apartment_complex_tensor, economy_tensor), dim=1)
        encoder_input_tensors[i] = encoder_input_tensor

    with torch.no_grad():
        dong_apartment_complexes_embedding_matrixes = torch.zeros(encoder_input_tensors.shape[0],encoder_input_tensors.shape[1],embedding_dim)
        for i in range(encoder_input_tensors.shape[0]):
            apartment_complex_embedding_matrix = torch.zeros(encoder_input_tensor.shape[0],embedding_dim)
            for j in range(encoder_input_tensor.shape[0]):
                apartment_complex_embedding_vector = model.encoder(encoder_input_tensor[i].unsqueeze(0)).squeeze()
                apartment_complex_embedding_matrix[i] = apartment_complex_embedding_vector
            dong_apartment_complexes_embedding_matrixes[i] = apartment_complex_embedding_matrix

    dong_apartment_complexes_aids = table_1[table_1['dong'] == dong]['aid'].values
    dong_apartment_complexes_prices = torch.zeros(encoder_input_tensors.shape[0],encoder_input_tensors.shape[1],1)
    for i, dong_apartment_complex_aid in zip(range(encoder_input_tensors.shape[0]),dong_apartment_complexes_aids):
        dong_apartment_complexes_prices[i] = torch.from_numpy(pd.DataFrame({'did': range(0, 204)}).merge(table_3[table_3['aid'] == dong_apartment_complex_aid][['did','price']], on='did', how='outer').fillna(0).set_index('did').values)

    dong_apartment_complexes_embedding_matrix_with_window_size = torch.zeros(204-lstm_window_size,max_dong_apartment_complexes,lstm_window_size,embedding_dim)
    dong_apartment_complexes_prices_with_window_size = torch.zeros(204-lstm_window_size,max_dong_apartment_complexes,1)

    break

    # for i in range(dong_apartment_complexes_embedding_matrix_with_window_size.shape[0]):
    #     for j in range(dong_apartment_complexes_embedding_matrixes.shape[0]):
    #         dong_apartment_complexes_embedding_matrix_with_window_size[i][j] = dong_apartment_complexes_embedding_matrixes[j][i:i+lstm_window_size,:]
    #         dong_apartment_complexes_price_with_window_size[i][j] = 
    # apartment_complex_embedding_vector = model.encoder(encoder_input_tensor[i].unsqueeze(0)).squeeze().numpy()
    # apartment_complex_embedding_matrix[i] = apartment_complex_embedding_vector
    


torch.Size([4, 204, 1])
